In [1]:
import tensorflow as tf
import tensorflow_recommenders as tfrs
import numpy as np
import pandas as pd
from tensorflow.keras.callbacks import EarlyStopping
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from collections import Counter
import os
import re
import ast

In [2]:
recipe_df = pd.read_csv("Food.csv", nrows=1000)

In [3]:
recipe_df

,Unnamed: 0,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...,...,...
995,995,Winter of Our Content,"['3/4 ounce Laird’s Straight Apple Brandy', '3...","Combine the brandy, lemon juice, maple syrup, ...",winter-of-our-content-apple-brandy-sparkling-w...,"['3/4 ounce Laird’s Straight Apple Brandy', '3..."
996,996,Perfect Circle,['1 1/2 cups chilled fino or manzanilla sherry...,"Up to 24 hours before serving, make the batch....",perfect-circle-champagne-sherry-campari-cocktail,['1 1/2 cups chilled fino or manzanilla sherry...
997,997,All She Wrote,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ...","At least 2 hours before serving, make the batc...",all-she-wrote-punt-e-mes-vermouth-cocktail,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ..."
998,998,Mr. Tingles' Punch,"['1 (750 ml) bottle light rum', '2 tablespoons...",At least 24 hours before you plan to serve the...,mr-tingles-pomegranate-rum-punch,"['1 (750 ml) bottle light rum', '2 tablespoons..."


In [4]:
recipe_df = recipe_df.drop(columns=["Unnamed: 0", "Image_Name"])
recipe_df

,Title,Ingredients,Instructions,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...","['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
...,...,...,...,...
995,Winter of Our Content,"['3/4 ounce Laird’s Straight Apple Brandy', '3...","Combine the brandy, lemon juice, maple syrup, ...","['3/4 ounce Laird’s Straight Apple Brandy', '3..."
996,Perfect Circle,['1 1/2 cups chilled fino or manzanilla sherry...,"Up to 24 hours before serving, make the batch....",['1 1/2 cups chilled fino or manzanilla sherry...
997,All She Wrote,"['2 1/4 cups chilled Punt e Mes', '1 cup plus ...","At least 2 hours before serving, make the batc...","['2 1/4 cups chilled Punt e Mes', '1 cup plus ..."
998,Mr. Tingles' Punch,"['1 (750 ml) bottle light rum', '2 tablespoons...",At least 24 hours before you plan to serve the...,"['1 (750 ml) bottle light rum', '2 tablespoons..."


In [5]:
print(recipe_df['Cleaned_Ingredients'].head(5).to_string(index=False))

['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher ...
['2 large egg whites', '1 pound new potatoes (a...
['1 cup evaporated milk', '1 cup whole milk', '...
['1 (¾- to 1-pound) round Italian loaf, cut int...
['1 teaspoon dark brown sugar', '1 teaspoon hot...


In [6]:
def clean_and_parse_list_str(x):
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
            else:
                return [str(parsed)]
        except (ValueError, SyntaxError):
            return [x]
    elif isinstance(x, list):
        return x
    else:
        return []

def clean_ingredient_list(ingredient_list):
    cleaned = []
    garbage_words = {
        'about', 'plus', 'divided', 'total', 'such as', 'room temperature',
        'to', 'into', 'pieces', 'torn', 'cut', 'cored', 'more', 'melted'
    }
    
    for item in ingredient_list:
        # Hilangkan tanda strip di awal
        part = item.lstrip('-').strip()

        # Pisah berdasarkan koma, karena di bahan mungkin ada beberapa bahan dipisah koma
        parts = [p.strip() for p in part.split(',') if p.strip()]
        
        for p in parts:
            # Bersihkan angka, pecahan, satuan, kata tidak penting, karakter aneh
            p = re.sub(r'[\d¼½¾⅓⅔⅛⅜⅝⅞\(\)\.\–\-]+', '', p)
            p = re.sub(r'\b(tsp|tbsp|cups?|oz|lb|lbs|g|kg|ml|l|inch|inches)\b', '', p, flags=re.IGNORECASE)
            
            for word in garbage_words:
                p = re.sub(rf'\b{word}\b', '', p, flags=re.IGNORECASE)
            
            p = re.sub(r'[^a-zA-Z\s]', '', p)
            p = re.sub(r'\s+', ' ', p).strip()
            
            if p:
                cleaned.append(p)
    
    return cleaned

# Terapkan ke dataframe
recipe_df["Ingredients"] = recipe_df["Ingredients"].apply(clean_and_parse_list_str)
recipe_df["Cleaned_Ingredients"] = recipe_df["Ingredients"].apply(clean_ingredient_list)
recipe_df

,Title,Ingredients,Instructions,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[1 (3½–4-lb.) whole chicken, 2¾ tsp. kosher sa...","Pat chicken dry with paper towels, season all ...","[whole chicken, kosher salt, small acorn squas..."
1,Crispy Salt and Pepper Potatoes,"[2 large egg whites, 1 pound new potatoes (abo...",Preheat oven to 400°F and line a rimmed baking...,"[large egg whites, pound new potatoes in diame..."
2,Thanksgiving Mac and Cheese,"[1 cup evaporated milk, 1 cup whole milk, 1 ts...",Place a rack in middle of oven; preheat to 400...,"[evaporated milk, whole milk, garlic powder, o..."
3,Italian Sausage and Bread Stuffing,"[1 (¾- to 1-pound) round Italian loaf, cut int...",Preheat oven to 350°F with rack in middle. Gen...,"[pound round Italian loaf, cubes, tablespoons ..."
4,Newton's Law,"[1 teaspoon dark brown sugar, 1 teaspoon hot w...",Stir together brown sugar and hot water in a c...,"[teaspoon dark brown sugar, teaspoon hot water..."
...,...,...,...,...
995,Winter of Our Content,"[3/4 ounce Laird’s Straight Apple Brandy, 3/4 ...","Combine the brandy, lemon juice, maple syrup, ...","[ounce Lairds Straight Apple Brandy, ounce lem..."
996,Perfect Circle,"[1 1/2 cups chilled fino or manzanilla sherry,...","Up to 24 hours before serving, make the batch....","[chilled fino or manzanilla sherry, Campari, b..."
997,All She Wrote,"[2 1/4 cups chilled Punt e Mes, 1 cup plus 2 t...","At least 2 hours before serving, make the batc...","[chilled Punt e Mes, tablespoons dry vermouth ..."
998,Mr. Tingles' Punch,"[1 (750 ml) bottle light rum, 2 tablespoons Si...",At least 24 hours before you plan to serve the...,"[bottle light rum, tablespoons Sichuan pepperc..."


In [7]:
recipe_df.to_csv("Food_Cleaned.csv")

In [8]:
# Buat user dan resep
recipes = recipe_df["Title"].unique()
num_users = 1000
user_ids = [f"user_{i+1}" for i in range(num_users)]

# Bangun interactions
interactions = []
np.random.seed(42)

for user in user_ids:
    liked = np.random.choice(recipes, size=50, replace=False)
    for recipe in liked:
        interactions.append({"user_id": user, "title": recipe})

interaction_df = pd.DataFrame(interactions)

# Atur ulang seed untuk rating
np.random.seed(42)

# Jumlah total interaksi
n = len(interaction_df)

# Buat 80% rating > 3 (yaitu: 4 atau 5)
high_ratings = np.random.randint(4, 6, size=int(n * 0.9))  # 4 atau 5
# Buat 20% rating <= 3 (1, 2, atau 3)
low_ratings = np.random.randint(1, 4, size=n - len(high_ratings))

# Gabungkan dan acak
all_ratings = np.concatenate([high_ratings, low_ratings])
np.random.shuffle(all_ratings)

# Masukkan ke DataFrame
interaction_df["rating"] = all_ratings

# Tampilkan tabel
interaction_df

,user_id,title,rating
0,user_1,Pork Katsu Sandwich,1
1,user_1,Ice Water Salad,4
2,user_1,Skirt Steak with Spicy Coconut Dressing,5
3,user_1,Tomato-Watermelon Salad with Turmeric Oil,4
4,user_1,Shredded Daikon Salad,5
...,...,...,...
49995,user_1000,Shaved Carrots with Charred Dates,4
49996,user_1000,"Sumac, Spelt, and Apple Cake",4
49997,user_1000,Coconut Rice Pudding Pie,1
49998,user_1000,Sloppy Joe Shirred Eggs With Spinach,2


In [9]:
# ====================
# 1. Data Preprocessing
# ====================
interaction_df["user_id"] = interaction_df["user_id"].astype(str)
interaction_df["title"] = interaction_df["title"].astype(str)

unique_user_ids = interaction_df["user_id"].unique()
unique_titles = interaction_df["title"].unique()

train_df, test_df = train_test_split(interaction_df, test_size=0.2, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": train_df["user_id"].values,
    "title": train_df["title"].values,
    "rating": train_df["rating"].values
}).shuffle(100).batch(256).cache()

val_ds = tf.data.Dataset.from_tensor_slices({
    "user_id": test_df["user_id"].values,
    "title": test_df["title"].values,
    "rating": test_df["rating"].values
}).batch(512)

# ====================
# 2. Lookup Layers
# ====================
user_lookup = tf.keras.layers.StringLookup(vocabulary=unique_user_ids, mask_token=None)
title_lookup = tf.keras.layers.StringLookup(vocabulary=unique_titles, mask_token=None)

# ====================
# 3. User dan Item Model
# ====================
embedding_dim = 512

class UserModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.model = tf.keras.Sequential([
            user_lookup,
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])
    def call(self, inputs):
        return self.model(inputs)

class ItemModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.model = tf.keras.Sequential([
            title_lookup,
            tf.keras.layers.Embedding(len(unique_titles) + 1, embedding_dim),
            tf.keras.layers.Dense(1024, activation="relu"),
            tf.keras.layers.Dense(embedding_dim)
        ])
    def call(self, inputs):
        return self.model(inputs)

# ====================
# 4. Ranking Model
# ====================
class RecipeModel(tfrs.Model):
    def __init__(self, user_model, item_model):
        super().__init__()
        self.user_model = user_model
        self.item_model = item_model
        self.rating_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.MeanSquaredError(),
            metrics=[tf.keras.metrics.RootMeanSquaredError()]
        )
    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model(features["title"])
        rating_predictions = tf.reduce_sum(user_embeddings * item_embeddings, axis=1)
        return self.rating_task(labels=features["rating"], predictions=rating_predictions)
    def call(self, features):
        user_embeddings = self.user_model(features["user_id"])
        item_embeddings = self.item_model(features["title"])
        return tf.reduce_sum(user_embeddings * item_embeddings, axis=1)

# ====================
# 5. Latih Model
# ====================
user_model = UserModel()
item_model = ItemModel()
model = RecipeModel(user_model, item_model)

model.compile(optimizer=tf.keras.optimizers.Adam(0.0005))
early_stop = EarlyStopping(monitor="root_mean_squared_error", patience=10, restore_best_weights=True)

model.fit(train_ds, validation_data=val_ds, epochs=100, callbacks=[early_stop])

# ====================
# 6. Simpan Model
# ====================
os.makedirs("saved_model", exist_ok=True)
# 1. Dummy input agar model 'build'
_ = model({
    "user_id": tf.constant(["dummy_user"]),
    "title": tf.constant(["dummy_title"]),
    "rating": tf.constant([5.0])
})

model.save("saved_model/user_model_v1")

# ====================
# 7. Buat BruteForce Index
# ====================
index = tfrs.layers.factorized_top_k.BruteForce(user_model)

candidates_ds = tf.data.Dataset.from_tensor_slices(unique_titles).batch(128)
candidates_ds = candidates_ds.map(lambda title: (title, item_model(title)))

index.index_from_dataset(candidates_ds)

# ====================
# 8. Simpan BruteForce Index
# ====================
class RecommendationIndex(tf.Module):
    def __init__(self, index_layer):
        super().__init__()
        self.index_layer = index_layer

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None], dtype=tf.string),
        tf.TensorSpec(shape=(), dtype=tf.int32)
    ])
    def recommend(self, user_ids, k):
        return self.index_layer(user_ids, k=k)

recommender = RecommendationIndex(index)
tf.saved_model.save(recommender, "saved_model/brute_force_index_v1")


Epoch 1/100
157/157 [==============================] - 4s 12ms/step - root_mean_squared_error: 1.4474 - loss: 2.0802 - regularization_loss: 0.0000e+00 - total_loss: 2.0802 - val_root_mean_squared_error: 1.0171 - val_loss: 0.8962 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.8962
Epoch 2/100
157/157 [==============================] - 2s 10ms/step - root_mean_squared_error: 0.9750 - loss: 0.9466 - regularization_loss: 0.0000e+00 - total_loss: 0.9466 - val_root_mean_squared_error: 1.0264 - val_loss: 0.9274 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.9274
Epoch 3/100
157/157 [==============================] - 2s 10ms/step - root_mean_squared_error: 0.9500 - loss: 0.8980 - regularization_loss: 0.0000e+00 - total_loss: 0.8980 - val_root_mean_squared_error: 1.0251 - val_loss: 0.9484 - val_regularization_loss: 0.0000e+00 - val_total_loss: 0.9484
Epoch 4/100
157/157 [==============================] - 2s 10ms/step - root_mean_squared_error: 0.9324 - loss: 0.8648 - regul

INFO:tensorflow:Assets written to: saved_model/user_model_v1\assets


INFO:tensorflow:Assets written to: saved_model/user_model_v1\assets


INFO:tensorflow:Assets written to: saved_model/brute_force_index_v1\assets


INFO:tensorflow:Assets written to: saved_model/brute_force_index_v1\assets


In [10]:
user_id = "user_3"
user_interactions = interaction_df[interaction_df["user_id"] == user_id]

print(f"Resep yang telah diberi rating oleh user {user_id}:")
for _, row in user_interactions.iterrows():
    print(f"- {row['title']} (Rating: {row['rating']})")

visited_titles = set(user_interactions["title"].values)

# Panggil BruteForce index dengan input user_id, rekomendasi dari top k
results = index(tf.constant([user_id]))
scores, titles = results  # unpack tuple

scores = scores.numpy()[0]
titles = titles.numpy()[0]

print(f"\nRekomendasi untuk user {user_id} (resep yang belum di-rate):")
count = 0
max_results = 20

for score, title in zip(scores, titles):
    title_str = title.decode("utf-8")
    if title_str in visited_titles:
        continue
    print(f"- {title_str} (Skor: {score:.4f})")
    count += 1
    if count >= max_results:
        break

if count == 0:
    print("Tidak ada rekomendasi resep baru untuk ditampilkan.")
elif count < max_results:
    print(f"(Hanya {count} resep baru yang bisa direkomendasikan)")

Resep yang telah diberi rating oleh user user_3:
- New-and-Improved Greek Salad (Rating: 4)
- Squash and Radicchio Salad With Pecans (Rating: 4)
- Oaxaca Old Fashioned (Rating: 5)
- Za'atar Spice Blend (Rating: 5)
- Buttermilk Cornbread (Rating: 5)
- Butternut Squash, Coconut, and Ginger Muffins (Rating: 4)
- Tomato and Parmesan Risotto (Rating: 4)
- Camouflage Chocolate Fudge Brownies (Rating: 2)
- Fragrant Mixed Herb and Flatbread Salad (Domaaj) (Rating: 1)
- Tortillas de Harina (Flour Tortillas) (Rating: 5)
- Spaghetti with No-Cook Puttanesca (Rating: 5)
- Chopped BLT Salad (Rating: 4)
- Double Pecan Thumbprints (Rating: 4)
- Best Deviled Eggs (Rating: 4)
- Shrimp Ramp-y (Rating: 4)
- Jerk Baby Back Ribs With Pineapple Salsa (Rating: 4)
- Pork Volcánes al Pastor (Rating: 5)
- Instant Pot Lamb Haleem (Rating: 4)
- Torn Potatoes of Many Colors With Chile-Lime Butter (Rating: 5)
- Miso-Squash Ramen (Rating: 4)
- Big Green Lentil Salad (Rating: 3)
- Roasted Cauliflower with Parmesan-Pan

In [11]:
# ====================
# Data dan variabel awal
# ====================
item_popularity = Counter(interaction_df["title"])
total_interaction = sum(item_popularity.values())
all_items = set(interaction_df["title"].unique())

user_history = interaction_df.groupby("user_id")["title"].apply(set).to_dict()

# ====================
# Precision@K
# ====================
def precision_at_k(k):
    correct = 0
    total = 0
    test_dict = test_df.groupby("user_id")["title"].apply(set).to_dict()

    for user_id, true_items in test_dict.items():
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_ids = set(title.numpy().decode("utf-8") for title in recommended[0])
        hits = recommended_ids.intersection(true_items)
        correct += len(hits)
        total += k
    return correct / total if total > 0 else 0

# ====================
# Novelty@K
# ====================
def novelty_at_k(k):
    total_novelty = 0.0
    total_users = 0

    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        novelty_score = 0.0
        for title in recommended[0].numpy():
            title_str = title.decode("utf-8")
            freq = item_popularity[title_str] / total_interaction
            if freq > 0:
                novelty_score += -np.log2(freq)
        avg_novelty = novelty_score / k
        total_novelty += avg_novelty
        total_users += 1

    return total_novelty / total_users if total_users > 0 else 0

# ====================
# Coverage@K
# ====================
def coverage_at_k(k):
    recommended_items = set()
    for user_id in test_df["user_id"].unique():
        _, recommended = index(tf.constant([user_id]), k=k)
        for title in recommended[0].numpy():
            recommended_items.add(title.decode("utf-8"))
    return len(recommended_items) / len(all_items)

# ============================
# Serendipity Precomputation
# ============================
title_to_embedding = {}
for title in interaction_df["title"].unique():
    vec = model.item_model(tf.constant([title]))
    vec = tf.math.l2_normalize(vec, axis=1)
    title_to_embedding[title] = vec.numpy()[0]

user_mean_embeddings = {}
for user_id in test_df["user_id"].unique():
    seen = user_history.get(user_id, set())
    seen_vecs = [title_to_embedding[title] for title in seen if title in title_to_embedding]
    if not seen_vecs:
        continue
    seen_avg = np.mean(seen_vecs, axis=0)
    seen_avg = seen_avg / np.linalg.norm(seen_avg)
    user_mean_embeddings[user_id] = seen_avg

# ============================
# Serendipity@K
# ============================
def make_serendipity_function(k):
    def compute_serendipity_batch(user_id_tensor):
        user_id = user_id_tensor.numpy().decode("utf-8") if isinstance(user_id_tensor.numpy(), bytes) else str(user_id_tensor.numpy())

        seen = user_history.get(user_id, set())
        if not seen or user_id not in user_mean_embeddings:
            return tf.constant(0.0, dtype=tf.float32)

        seen_avg = user_mean_embeddings[user_id].reshape(1, -1)
        _, recommended = index(tf.constant([user_id]), k=k)
        recommended_titles = recommended[0].numpy()

        ser = 0
        valid_k = 0
        for title in recommended_titles:
            title_str = title.decode("utf-8")
            if title_str in seen or title_str not in title_to_embedding:
                continue
            rec_vec = title_to_embedding[title_str].reshape(1, -1)
            dot_sim = np.dot(seen_avg, rec_vec.T)[0][0]
            ser += 1 - dot_sim
            valid_k += 1

        score = ser / valid_k if valid_k > 0 else 0.0
        return tf.constant(score, dtype=tf.float32)

    return compute_serendipity_batch

def serendipity_at_k(k):
    compute_fn = make_serendipity_function(k)
    user_ids_tensor = tf.constant(list(user_mean_embeddings.keys()), dtype=tf.string)

    serendipity_scores = tf.data.Dataset.from_tensor_slices(user_ids_tensor) \
        .map(lambda uid: tf.py_function(func=compute_fn, inp=[uid], Tout=tf.float32)) \
        .batch(128) \
        .map(lambda batch: tf.reduce_sum(batch)) \
        .reduce(tf.constant(0.0), lambda acc, val: acc + val)

    return serendipity_scores.numpy() / len(user_mean_embeddings)

# ============================
# Evaluasi Semua Metrik
# ============================
def evaluate_model(k_values=[10]):
    print("Evaluasi sistem rekomendasi:")
    for k in k_values:
        print(f"\nTop-{k}")
        precision = precision_at_k(k)
        novelty = novelty_at_k(k)
        coverage = coverage_at_k(k)
        serendipity = serendipity_at_k(k)
        print(f"- Precision@{k}:   {precision:.4f}")
        print(f"- Novelty@{k}:     {novelty:.4f}")
        print(f"- Coverage@{k}:    {coverage:.4f}")
        print(f"- Serendipity@{k}: {serendipity:.4f}")

# ============================
# Jalankan Evaluasi
# ============================
evaluate_model(k_values=[10, 50, 100])

Evaluasi sistem rekomendasi:

Top-10
- Precision@10:   0.0093
- Novelty@10:     9.9638
- Coverage@10:    0.7270
- Serendipity@10: 0.2963

Top-50
- Precision@50:   0.0095
- Novelty@50:     9.9656
- Coverage@50:    0.9620
- Serendipity@50: 0.2908

Top-100
- Precision@100:   0.0098
- Novelty@100:     9.9685
- Coverage@100:    0.9910
- Serendipity@100: 0.2885


In [12]:
# Load index
index = tf.saved_model.load("saved_model/brute_force_index_v1")

# Contoh user id
user_ids = tf.constant(["user_1"]) 

# Panggil fungsi recommend dari index dengan parameter k = jumlah top rekomendasi
scores, titles = index.recommend(user_ids, k=25)

for i, user_id in enumerate(user_ids.numpy()):
    print(f"Rekomendasi untuk user_id = {user_id.decode('utf-8')}:")
    for score, title in zip(scores[i].numpy(), titles[i].numpy()):
        print(f"- {title.decode('utf-8')} (score: {score:.4f})")
    print()


Rekomendasi untuk user_id = user_1:
- Shirley Tonic (score: 6.7224)
- Spiced Lentil and Caramelized Onion Baked Eggs (score: 6.3598)
- Spanish Penny (score: 6.2756)
- Cacao Water (Agua de Cacao) (score: 6.2603)
- Glazed and Flaky Apple Tart (score: 6.2323)
- Sheet-Pan Chicken Meatballs and Charred Broccoli (score: 6.1170)
- Killer Chocolate Cake (score: 6.1105)
- Coconut-Fig Energy Balls (score: 6.0981)
- Raspberry Rugelach (score: 6.0930)
- No-Churn Fresh Mint and Chocolate Ice Cream (score: 6.0590)
- Salt-and-Pepper Fish (score: 6.0325)
- Dry-Rubbed Turkey Breast (score: 6.0212)
- Earl Grey Doughnuts with Brown Butter Glaze (score: 6.0112)
- Sheet-Pan Potato Hash with Fixins (score: 5.9594)
- Grilled Salt-and-Pepper Black Bass with Curry Verde (score: 5.9474)
- Labne Deviled Eggs with Paprika and Ginger (score: 5.9363)
- Shingled Sweet Potatoes with Harissa (score: 5.9348)
- Sweet Potato Bowls With Kale and Chickpeas (score: 5.9329)
- Skillet Dressing with Cornbread and Biscuits (sco